In [1]:
# Install dependencies
!pip install --upgrade pytubefix openai-whisper moviepy tinytag scenedetect


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 30.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 768.9/768.9 kB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.9/130.9 kB 12.2 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=8c3f3bc77eed974341d3dd3dda28e1978a4ffc2f577f1cc375107781c156526e
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper
  Attempting uninstall: moviepy
    Found existing installation: moviepy 1.0.3
    Uninstalling moviepy-1.0.3:
      Successfully uninstalled moviepy-1.0.3


In [4]:
from pytubefix import YouTube
from pytubefix.cli import on_progress
import whisper
#from moviepy.editor import VideoFileClip
from tinytag import TinyTag
from scenedetect import open_video, detect, ContentDetector
import os


In [5]:
def download_youtube_video(url: str, output_path: str = "Videos/") -> str:
    yt = YouTube(url, on_progress_callback=on_progress)
    video = yt.streams.filter(progressive=True, file_extension="mp4").first()
    os.makedirs(output_path, exist_ok=True)
    file_path = video.download(output_path)
    print(f"Downloaded: {file_path}")
    return file_path


In [6]:
def transcribe_video(video_path: str, model_name: str = "base") -> str:
    model = whisper.load_model(model_name)
    result = model.transcribe(video_path)
    print("Transcription complete.")
    return result["text"]


In [7]:
def extract_audio(video_path: str, output_audio: str = "extracted_audio.mp3") -> str:
    video = VideoFileClip(video_path)
    audio = video.audio
    audio.write_audiofile(output_audio)
    print(f"Audio saved: {output_audio}")
    return output_audio


In [8]:
def get_metadata(video_path: str):
    tag = TinyTag.get(video_path)
    metadata = {
        "title": tag.title,
        "artist": tag.artist,
        "duration (s)": tag.duration,
        "filesize (bytes)": tag.filesize,
    }
    return metadata


In [9]:
def detect_scenes(video_path: str):
    video = open_video(video_path)
    scenes = detect(video, ContentDetector())
    scene_list = [(str(scene[0]), str(scene[1])) for scene in scenes]
    print("Scenes detected:")
    for s in scene_list:
        print(f"{s[0]} - {s[1]}")
    return scene_list
